In [29]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras import Model

# Load the pre-trained model (EfficientNetB0 in this case)
model = tf.keras.applications.EfficientNetB0(weights='imagenet', include_top=True)

# Step 1: Preprocess the image
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))  
    img_array = image.img_to_array(img)  
    img_array = np.expand_dims(img_array, axis=0) 
    img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)  
    return img, img_array

# Step 2: Grad-CAM function
def grad_cam(input_model, img_array, layer_name="block6a_activation"):
    grad_model = Model([input_model.inputs], [input_model.get_layer(layer_name).output, input_model.output])
    
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, np.argmax(predictions[0])]
        
    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))  # Average over all spatial locations
    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_mean(conv_outputs * pooled_grads, axis=-1).numpy()  # Weighted sum of feature maps
    
    # Normalize the heatmap
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    
    return heatmap

# Step 3: Overlay the heatmap on the image
def overlay_heatmap(img, heatmap, alpha=0.4):
    heatmap = np.uint8(255 * heatmap)
    heatmap = np.expand_dims(heatmap, axis=-1)
    heatmap_resized = tf.image.resize(heatmap, (img.size[1], img.size[0])).numpy()
    heatmap_resized = np.squeeze(heatmap_resized)
    
    # Apply a colormap
    colormap = plt.cm.viridis(heatmap_resized / 255.0)[:, :, :3] * 255
    colormap = np.uint8(colormap)
    
    # Blend the original image with the colormap
    overlayed_img = np.array(img) * (1 - alpha) + colormap * alpha
    overlayed_img = np.uint8(overlayed_img)
    
    return overlayed_img

# Step 4: Display Results
test_image_path = "/kaggle/input/skinburn/archive/skin burn dataset/test/img1045_jpg.rf.54585608065210f034be42784820f60d.jpg"
original_img, processed_img = preprocess_image(test_image_path)

# Generate heatmap
heatmap = grad_cam(model, processed_img)

# Overlay heatmap on the original image
overlayed_img = overlay_heatmap(original_img, heatmap)

# Step 5: Plot the images (original, heatmap, and overlayed image)
plt.figure(figsize=(10, 5))

# Plot original image
plt.subplot(1, 3, 1)
plt.title("Original Image")
plt.imshow(original_img)
plt.axis('off')

# Plot Grad-CAM heatmap
plt.subplot(1, 3, 2)
plt.title("Grad-CAM Heatmap")
plt.imshow(heatmap, cmap='viridis')
plt.axis('off')

# Plot Overlayed Image
plt.subplot(1, 3, 3)
plt.title("Overlayed Image")
plt.imshow(overlayed_img)
plt.axis('off')

plt.show()


KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetB0
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Paths
train_dir = "/kaggle/input/degree/skin burn dataset/train"
val_dir = "/kaggle/input/degree/skin burn dataset/valid"

# Parameters
batch_size = 32
img_height = 224
img_width = 224
num_classes = 4
epochs = 15  

# Data Generators with Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,  
    brightness_range=[0.8, 1.2],  
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Calculate Class Weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weight_dict = dict(enumerate(class_weights))

# Model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False  
model = models.Sequential([
    layers.InputLayer(input_shape=(img_size, img_size, 3)),  # Explicit input shape
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])




# Learning Rate Scheduler: Exponential Decay
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,  # Starting learning rate
    decay_steps=100000,  # Number of steps before decaying the learning rate
      # Decay rate (learning rate will decrease by 96% every decay_steps)
    staircase=True  # Applies decay at discrete intervals
)


# Fine-tuning EfficientNetB0 (unfreeze layers)
base_model.trainable = True  

# Specify the layer from which to start fine-tuning
fine_tune_at = 100  # Example: Unfreeze layers from index 100 onward

# Freeze all layers up to 'fine_tune_at'
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Print the model summary to verify the layers' trainability status
model.summary()
# Compile the model with the learning rate schedule
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr]
)



In [ ]:
!pip install --upgrade tensorflow

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt

# Set parameters
img_size = 224
batch_size = 32
epochs = 50

# Paths to dataset
data_dir = '/kaggle/input/data-skin/skin burn dataset/train'
val_dir = '/kaggle/input/data-skin/skin burn dataset/valid'

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

# Debug shapes
for data_batch, labels_batch in train_generator:
    print("Data batch shape:", data_batch.shape)
    print("Labels batch shape:", labels_batch.shape)
    break

# Model building
num_classes = len(train_generator.class_indices)
print("Number of classes:", num_classes)
print("Class indices:", train_generator.class_indices)

# Define the input layer explicitly
input_tensor = layers.Input(shape=(img_size, img_size, 3))

# Load EfficientNetB0 base model without the top layer (classification layer)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Unlock all layers of EfficientNetB0 for training
base_model.trainable = True

# Optionally, freeze some layers of the EfficientNetB0 model
# You can choose to freeze the first few layers for better generalization
# For example, freeze all layers except the last 10 layers
# for layer in base_model.layers[:-10]:  
#     layer.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])



# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Lower learning rate for fine-tuning
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr]
)

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss')

plt.show()

# Save the model
model.save('efficientnetb0_skin_burn_classification.h5')


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 2.5 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 100.6 MB/s eta 0:00:0000:01
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.0
    Uninstalling tensorboard-2.17.0:
      Successfully uninstalled tensorboard-2.17.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.0
    Uninstalling tensorflow-2.17.0:
      Successfully uninstalled tensorflow-2.17.0


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Model Summary
model.summary()

# Save Model and Weights Separately
model.save('efficientnet_skin_burn_model_finetuned.h5')
# Save Weights with Correct Filename Extension
model.save_weights('efficientnet_skin_burn_model_weights.weights.h5')


# Visualization of Training History
def plot_training_history(history):
    # Accuracy
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

plot_training_history(history)

# Classification Report
val_generator.reset()  # Ensure the generator starts from the beginning
y_true = val_generator.classes
y_pred = np.argmax(model.predict(val_generator), axis=-1)

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=list(val_generator.class_indices.keys())))
